# Clone repository 

In [ ]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 78360, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 78360 (delta 70), reused 130 (delta 54), pack-reused 78197
Receiving objects: 100% (78360/78360), 593.59 MiB | 42.40 MiB/s, done.
Resolving deltas: 100% (55693/55693), done.


# Installing protobuff

In [ ]:
!cd models/research/ && protoc object_detection/protos/*.proto --python_out=.

# Install python API 

In [ ]:
!cd models/research && cp object_detection/packages/tf2/setup.py . && python -m pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 11.0 MB 39.8 MB/s 
     |████████████████████████████████| 352 kB 70.3 MB/s 
     |████████████████████████████████| 2.2 MB 58.6 MB/s 
     |████████████████████████████████| 25.0 MB 1.2 MB/s 
     |████████████████████████████████| 67 kB 6.1 MB/s 
     |████████████████████████████████| 116 kB 96.4 MB/s 
     |████████████████████████████████| 1.3 MB 77.8 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |██

# Testing installation 

In [ ]:
!cd models/research && python object_detection/builders/model_builder_tf2_test.py 

2022-10-31 23:51:16.466348: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-31 23:51:16.634238: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-31 23:51:17.537286: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-10-31 23:51:17.537431: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinf

# Importing the needed libraries 

In [ ]:
import os 
import imageio
import pathlib
import tensorflow as tf 

import time 
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

import numpy as np 
from PIL import Image 
import matplotlib.pyplot as plt 
%matplotlib inline 

In [ ]:
# Enable GPU dynamic memory allocation 
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus : 
    tf.confi.experimental.set_memory_growth(gpu, True)

# Dowloading videos

In [ ]:
!wget -nc https://lazyprogrammer.me/cnn_class2_videos.zip

--2022-10-31 23:53:09--  https://lazyprogrammer.me/cnn_class2_videos.zip
Resolving lazyprogrammer.me (lazyprogrammer.me)... 104.21.23.210, 172.67.213.166, 2606:4700:3030::ac43:d5a6, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|104.21.23.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2073140 (2.0M) [application/zip]
Saving to: ‘cnn_class2_videos.zip’

cnn_class2_videos.z 100%[===================>]   1.98M  2.62MB/s    in 0.8s    

2022-10-31 23:53:11 (2.62 MB/s) - ‘cnn_class2_videos.zip’ saved [2073140/2073140]



In [ ]:
!unzip cnn_class2_videos.zip

Archive:  cnn_class2_videos.zip
  inflating: catdog.mp4              
  inflating: safari.mp4              
  inflating: traffic.mp4             


In [ ]:
!ls

catdog.mp4  cnn_class2_videos.zip  models  safari.mp4  sample_data  traffic.mp4


In [ ]:
INPUT_VIDEOS = ["catdog","safari","traffic"]

# Download and extract model files 

https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

In [ ]:
url ="http://download.tensorflow.org/models/object_detection/tf2/20200713/centernet_hg104_1024x1024_coco17_tpu-32.tar.gz"

PATH_TO_THE_MODEL_DIR = tf.keras.utils.get_file(
    fname="centernet_hg104_1024x1024_coco17_tpu-32",
    origin=url, 
    untar=True
)
PATH_TO_THE_MODEL_DIR

1424430731/1424430731 [==============================] - 9s 0us/step


'/root/.keras/datasets/centernet_hg104_1024x1024_coco17_tpu-32'

# Download label files 

In [ ]:
url = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/mscoco_label_map.pbtxt'

PATH_TO_LABELS = tf.keras.utils.get_file(
    fname="mscoco_label_map.pbtxt",
    origin=url,
    untar=False
)
PATH_TO_LABELS

5056/5056 [==============================] - 0s 0us/step


'/root/.keras/datasets/mscoco_label_map.pbtxt'

In [ ]:
!head {PATH_TO_LABELS}

item {
  name: "/m/01g317"
  id: 1
  display_name: "person"
}
item {
  name: "/m/0199g"
  id: 2
  display_name: "bicycle"
}


# Load the model

In [ ]:
PATH_TO_SAVED_MODEL=PATH_TO_THE_MODEL_DIR + "/saved_model"

In [ ]:
print("Loading model...", end="")
start_time=time.time()

#Loading the saved model and building the detection function : 
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time= time.time()

elapsed_time = end_time -start_time
print("Done, elapsed time : {} seconds".format(elapsed_time))


Loading model...

Done, elapsed time : 50.80011963844299 seconds


# Loading the labels

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap (
    PATH_TO_LABELS,
    use_display_name=True
)
category_index

{1: {'id': 1, 'name': 'person'},
 2: {'id': 2, 'name': 'bicycle'},
 3: {'id': 3, 'name': 'car'},
 4: {'id': 4, 'name': 'motorcycle'},
 5: {'id': 5, 'name': 'airplane'},
 6: {'id': 6, 'name': 'bus'},
 7: {'id': 7, 'name': 'train'},
 8: {'id': 8, 'name': 'truck'},
 9: {'id': 9, 'name': 'boat'},
 10: {'id': 10, 'name': 'traffic light'},
 11: {'id': 11, 'name': 'fire hydrant'},
 13: {'id': 13, 'name': 'stop sign'},
 14: {'id': 14, 'name': 'parking meter'},
 15: {'id': 15, 'name': 'bench'},
 16: {'id': 16, 'name': 'bird'},
 17: {'id': 17, 'name': 'cat'},
 18: {'id': 18, 'name': 'dog'},
 19: {'id': 19, 'name': 'horse'},
 20: {'id': 20, 'name': 'sheep'},
 21: {'id': 21, 'name': 'cow'},
 22: {'id': 22, 'name': 'elephant'},
 23: {'id': 23, 'name': 'bear'},
 24: {'id': 24, 'name': 'zebra'},
 25: {'id': 25, 'name': 'giraffe'},
 27: {'id': 27, 'name': 'backpack'},
 28: {'id': 28, 'name': 'umbrella'},
 31: {'id': 31, 'name': 'handbag'},
 32: {'id': 32, 'name': 'tie'},
 33: {'id': 33, 'name': 'suitc

# Object detection 

In [ ]:
def detect_objects_in_image (image_np) : 
   
    #The input needs to be a tensor, convert it using 'tf.convert_to_tensor'. 
    input_tensor= tf.convert_to_tensor(image_np)

    input_tensor= input_tensor[tf.newaxis,...]
    
    #Do the detection 
    detections = detect_fn(input_tensor)

    #All outputs are batches tensors 
    #Convert numpy arrays, and take index[0] to remove the batch dimension 
    #We are only interested in the first num_detections
    num_detections = int(detections.pop("num_detections"))
    detections={key: value[0,:num_detections].numpy() for key, value in detections.items()}
    detections["num_detections"]= num_detections

    #Detection classes should be ints : 
    detections["detection_classes"] = detections["detection_classes"].astype(np.int)


    image_np_with_detections= image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections, 
        detections["detection_boxes"],
        detections["detection_classes"],
        detections["detection_scores"],
        category_index, 
        use_normalized_coordinates =True, 
        max_boxes_to_draw=200,
        min_score_thresh=0.30, 
        agnostic_mode=False
    )

    return image_np_with_detections

In [ ]:
def detect_object_in_video(input_video): 
    print(f'Running inference for {input_video}.mpy...', end="")

    video_reader=imageio.get_reader(f'{input_video}.mp4')
    video_writer=imageio.get_writer(f'{input_video}_annotated.mp4',fps=10)

    #loop through and process each frame
    t0 = time.time()
    n_frames = 0 
    for frame in video_reader: 
        n_frames +=1
        new_frame= detect_objects_in_image(frame)

        #instead of plotting image, we write the frame to video 
        video_writer.append_data(new_frame)
    
    fps= n_frames/(time.time() - t0)
    print("Frames processed : %s, Speed : %s fps" % (n_frames, fps))

    #clean up 
    video_writer.close()
    

In [ ]:
!pip install imageio-ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 26.9 MB 1.3 MB/s 


In [ ]:
detect_object_in_video(INPUT_VIDEOS[0])

Running inference for catdog.mpy...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Frames processed : 50, Speed : 0.04031764163931248 fps


In [ ]:
detect_object_in_video(INPUT_VIDEOS[1])

Running inference for safari.mpy...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Frames processed : 100, Speed : 0.04116569955748983 fps


In [ ]:
detect_object_in_video(INPUT_VIDEOS[2])

Running inference for traffic.mpy...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Frames processed : 70, Speed : 0.0408769683956221 fps


the videos are saved in the colab files

# Testing to predict multiple images at one in the model